## Setting up Orchestrator Agent A2A

In the previous module, we've launched two agents, using AgentCore Runtime, that supports A2A invocations.

In this lab, we're going to add an orchestrator, that will invoke our sub-agents.

<img src="images/architecture.png" style="width: 80%;">

So let's get started!

### Setup

Import required dependencies

In [1]:
# Import libraries
import os
import json
import requests
import boto3
from boto3.session import Session
from strands.tools import tool

# Get boto session
boto_session = Session()
region = boto_session.region_name

Retrieve information from previous LABs, so we can use it during this one.

In [2]:
%store -r

### 1 - Create Code for the orchestrator agent

Let's generate Python code that will be used for our orchestrator, and lately will be deployed in AgentCore.

In [3]:
%%writefile agents/orchestrator.py
import logging
import json
import asyncio
from typing import Dict, Optional
from urllib.parse import quote
from uuid import uuid4

import httpx
from a2a.client import A2ACardResolver, ClientConfig, ClientFactory
from a2a.types import Message, Part, Role, TextPart

from helpers.utils import get_cognito_secret, reauthenticate_user, get_ssm_parameter, SSM_DOCS_AGENT_ARN, SSM_BLOGS_AGENT_ARN

from strands import Agent, tool
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from fastapi import HTTPException

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Reduced timeouts to prevent hanging
DEFAULT_TIMEOUT = 15  # 15s instead of 300s
AGENT_TIMEOUT = 10    # 10s per agent call

# Global cache and connection pool
_cache = {
    'cognito_config': None,
    'agent_arns': {},
    'agent_cards': {},
    'http_client': None
}

app = BedrockAgentCoreApp()

def get_cached_config():
    """Cache all expensive operations"""
    if not _cache['agent_arns']:
        _cache['agent_arns'] = {
            'docs': get_ssm_parameter(SSM_DOCS_AGENT_ARN),
            'blogs': get_ssm_parameter(SSM_BLOGS_AGENT_ARN)
        }
    
    if not _cache['cognito_config']:
        secret = json.loads(get_cognito_secret())
        _cache['cognito_config'] = {
            'client_id': secret.get("client_id"),
            'client_secret': secret.get("client_secret")
        }
    
    return _cache['agent_arns'], _cache['cognito_config']

def get_bearer_token():
    """Generate fresh bearer token for each request"""
    _, config = get_cached_config()
    return reauthenticate_user(
        config['client_id'], 
        config['client_secret']
    )

def get_http_client():
    """Reuse HTTP client with aggressive timeouts"""
    if not _cache['http_client']:
        _cache['http_client'] = httpx.AsyncClient(
            timeout=httpx.Timeout(DEFAULT_TIMEOUT, connect=5.0),
            limits=httpx.Limits(max_keepalive_connections=5, max_connections=10),
            http2=True  # Enable HTTP/2 for better performance
        )
    return _cache['http_client']

def create_message(text: str) -> Message:
    return Message(
        kind="message",
        role=Role.user,
        parts=[Part(TextPart(kind="text", text=text))],
        message_id=uuid4().hex,
    )

async def send_agent_message(message: str, agent_type: str) -> Optional[str]:
    """Optimized agent communication with circuit breaker pattern"""
    try:
        agent_arns, _ = get_cached_config()
        agent_arn = agent_arns[agent_type]
        bearer_token = get_bearer_token()
        
        from boto3.session import Session
        region = Session().region_name
        
        escaped_arn = quote(agent_arn, safe='')
        runtime_url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{escaped_arn}/invocations/"
        
        headers = {
            "Authorization": f"Bearer {bearer_token}",
            'X-Amzn-Bedrock-AgentCore-Runtime-Session-Id': str(uuid4())
        }
        
        httpx_client = get_http_client()
        httpx_client.headers.update(headers)
        
        # Cache agent card
        if agent_arn not in _cache['agent_cards']:
            resolver = A2ACardResolver(httpx_client=httpx_client, base_url=runtime_url)
            _cache['agent_cards'][agent_arn] = await asyncio.wait_for(
                resolver.get_agent_card(), timeout=5.0
            )
        
        agent_card = _cache['agent_cards'][agent_arn]
        
        # Create client with non-streaming mode
        config = ClientConfig(httpx_client=httpx_client, streaming=False)
        factory = ClientFactory(config)
        client = factory.create(agent_card)
        
        msg = create_message(message)
        
        # Use timeout for the entire operation
        async with asyncio.timeout(AGENT_TIMEOUT):
            async for event in client.send_message(msg):
                if isinstance(event, Message):
                    return event.parts[0].text if event.parts else "No response"
                elif isinstance(event, tuple) and len(event) == 2:
                    return event[0].parts[0].text if event[0].parts else "No response"
        
        return "Timeout: No response received"
        
    except asyncio.TimeoutError:
        logger.warning(f"Timeout calling {agent_type} agent")
        return f"Agent {agent_type} timed out"
    except Exception as e:
        logger.error(f"Error calling {agent_type}: {e}")
        return f"Error: {str(e)[:100]}"

@tool
async def send_mcp_message(message: str):
    """Send message to AWS Docs agent with timeout"""
    return await send_agent_message(f"Summarize briefly: {message}", 'docs')

@tool
async def send_blog_message(message: str):
    """Send message to AWS Blogs agent with timeout"""
    return await send_agent_message(f"Summarize briefly: {message}", 'blogs')


system_prompt = """You are an AWS information orchestrator. 

Available agents:
- AWS Documentation: Technical AWS service details
- AWS Blogs: Latest AWS news and announcements

IMPORTANT: Keep responses SHORT and FAST. Always request summaries from sub-agents.

Guidelines:
- Use parallel queries when possible
- Timeout after 10 seconds per agent
- Provide quick, actionable answers
- If agents timeout, provide what you know
"""

agent = Agent(
    system_prompt=system_prompt, 
    tools=[send_mcp_message, send_blog_message],
    name="AWS Orchestration Agent",
    description="An agent to orchestrate sub-agents"
)

@app.entrypoint
async def invoke_agent(payload, context):
    logger.info("Fast orchestrator processing request")
    
    try:
        user_prompt = payload.get("prompt", "")
        if not user_prompt:
            raise HTTPException(status_code=400, detail="No prompt provided")

        logger.info(f"Query: {user_prompt[:100]}...")
        
        # Set overall timeout for the entire operation
        async with asyncio.timeout(25.0):  # Max 25s total
            agent_stream = agent.stream_async(user_prompt)
            
            async for event in agent_stream:
                yield event

    except asyncio.TimeoutError:
        logger.error("Overall operation timed out")
        yield {"error": "Request timed out after 25 seconds"}
    except HTTPException:
        raise
    except Exception as e:
        logger.error(f"Processing failed: {e}")
        yield {"error": f"Processing failed: {str(e)[:100]}"}

# Cleanup on shutdown
async def cleanup():
    if _cache['http_client']:
        await _cache['http_client'].aclose()

if __name__ == "__main__":
    import atexit
    atexit.register(lambda: asyncio.run(cleanup()))
    app.run()

Overwriting agents/orchestrator.py


#### 1.1 - Create IAM Role for the Agent

In [4]:
from helpers.utils import create_agentcore_runtime_execution_role, ORCHESTRATOR_ROLE_NAME

agent_name="aws_orchestrator_assistant"

execution_role_arn = create_agentcore_runtime_execution_role(ORCHESTRATOR_ROLE_NAME)

ℹ️ Role AWSOrchestratorAssistantAgentCoreRole-us-east-1 already exists
Role ARN: arn:aws:iam::161615149547:role/AWSOrchestratorAssistantAgentCoreRole-us-east-1


### 2 - Deploy to AgentCore Runtime

Now, let's deploy the orchestrator in the AgentCore Runtime.

Note that in this example, we're not adding `protocol` parameter. Which means that this will be a HTTP agent.

In [5]:
from bedrock_agentcore_starter_toolkit import Runtime

agentcore_runtime = Runtime()

# Configure the deployment
response = agentcore_runtime.configure(
    entrypoint="agents/orchestrator.py",
    execution_role=execution_role_arn,
    auto_create_ecr=True,
    requirements_file="agents/requirements.txt",
    region=region,
    agent_name=agent_name,
    authorizer_configuration={
        "customJWTAuthorizer": {
            "allowedClients": [COGNITO_CLIENT_ID],
            "discoveryUrl": DISCOVERY_URL,
        }
    },
)

print("Configuration completed:", response)

Entrypoint parsed: file=/home/sagemaker-user/Multi-Agent-Collaboration/graph_IntelligentLoanUnderwriting/05-hosting-a2a/agents/orchestrator.py, bedrock_agentcore_name=orchestrator
Memory configured with STM only
Configuring BedrockAgentCore agent: aws_orchestrator_assistant


💡 No container engine found (Docker/Finch/Podman not installed)

✓ Default deployment uses CodeBuild (no container engine needed), For local builds, install Docker, Finch, or 
Podman

Will create new memory with mode: STM_ONLY
Memory configuration: Short-term memory only
Found existing memory ID from previous launch: aws_orchestrator_assistant_mem-qhOp4oEGM3


⚠️ Platform mismatch: Current system is 'linux/amd64' but Bedrock AgentCore requires 'linux/arm64', so local builds
won't work.
Please use default launch command which will do a remote cross-platform build using code build.For deployment other
options and workarounds, see: 
https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/getting-started-custom.html

Generated Dockerfile: Dockerfile
Generated .dockerignore: /home/sagemaker-user/Multi-Agent-Collaboration/graph_IntelligentLoanUnderwriting/05-hosting-a2a/.dockerignore
Keeping 'aws_orchestrator_assistant' as default agent
Bedrock AgentCore configured: /home/sagemaker-user/Multi-Agent-Collaboration/graph_IntelligentLoanUnderwriting/05-hosting-a2a/.bedrock_agentcore.yaml


Configuration completed: config_path=PosixPath('/home/sagemaker-user/Multi-Agent-Collaboration/graph_IntelligentLoanUnderwriting/05-hosting-a2a/.bedrock_agentcore.yaml') dockerfile_path=PosixPath('/home/sagemaker-user/Multi-Agent-Collaboration/graph_IntelligentLoanUnderwriting/05-hosting-a2a/Dockerfile') dockerignore_path=PosixPath('/home/sagemaker-user/Multi-Agent-Collaboration/graph_IntelligentLoanUnderwriting/05-hosting-a2a/.dockerignore') runtime='None' region='us-east-1' account_id='161615149547' execution_role='arn:aws:iam::161615149547:role/AWSOrchestratorAssistantAgentCoreRole-us-east-1' ecr_repository=None auto_create_ecr=True memory_id=None


In [6]:
launch_result = agentcore_runtime.launch()
print("Launch completed:", launch_result.agent_arn)

agent_arn = launch_result.agent_arn

🚀 CodeBuild mode: building in cloud (RECOMMENDED - DEFAULT)
   • Build ARM64 containers in the cloud with CodeBuild
   • No local Docker required
💡 Available deployment modes:
   • runtime.launch()                           → CodeBuild (current)
   • runtime.launch(local=True)                 → Local development
   • runtime.launch(local_build=True)           → Local build + cloud deploy (NEW)
Creating memory resource for agent: aws_orchestrator_assistant
✅ MemoryManager initialized for region: us-east-1
🔎 Retrieving memory resource with ID: aws_orchestrator_assistant_mem-qhOp4oEGM3...
  Found memory: aws_orchestrator_assistant_mem-qhOp4oEGM3
Found existing memory in cloud: aws_orchestrator_assistant_mem-qhOp4oEGM3
Existing memory has 0 strategies
✅ Using existing STM-only memory
Starting CodeBuild ARM64 deployment for agent 'aws_orchestrator_assistant' to account 161615149547 (us-east-1)
Setting up AWS resources (ECR repository, execution roles)...
Getting or creating ECR repository f

✅ Reusing existing ECR repository: 161615149547.dkr.ecr.us-east-1.amazonaws.com/bedrock-agentcore-aws_orchestrator_assistant


Getting or creating CodeBuild execution role for agent: aws_orchestrator_assistant
Role name: AmazonBedrockAgentCoreSDKCodeBuild-us-east-1-c5cab84b17
Reusing existing CodeBuild execution role: arn:aws:iam::161615149547:role/AmazonBedrockAgentCoreSDKCodeBuild-us-east-1-c5cab84b17
Using dockerignore.template with 45 patterns for zip filtering
Uploaded source to S3: aws_orchestrator_assistant/source.zip
Updated CodeBuild project: bedrock-agentcore-aws_orchestrator_assistant-builder
Starting CodeBuild build (this may take several minutes)...
Starting CodeBuild monitoring...
🔄 QUEUED started (total: 0s)
✅ QUEUED completed in 1.0s
🔄 PROVISIONING started (total: 1s)
✅ PROVISIONING completed in 8.2s
🔄 DOWNLOAD_SOURCE started (total: 9s)
✅ DOWNLOAD_SOURCE completed in 2.1s
🔄 BUILD started (total: 11s)
✅ BUILD completed in 16.4s
🔄 POST_BUILD started (total: 28s)
✅ POST_BUILD completed in 12.3s
🔄 COMPLETED started (total: 40s)
✅ COMPLETED completed in 1.0s
🎉 CodeBuild completed successfully in 0m

Launch completed: arn:aws:bedrock-agentcore:us-east-1:161615149547:runtime/aws_orchestrator_assistant-HVbqmWHSQJ


**Check Deployment Status**

Let's check if deployment is completed:

In [7]:
status_response = agentcore_runtime.status()
status = status_response.endpoint["status"]

print(f"Final status: {status}")

✅ MemoryManager initialized for region: us-east-1
🔎 Retrieving memory resource with ID: aws_orchestrator_assistant_mem-qhOp4oEGM3...
  Found memory: aws_orchestrator_assistant_mem-qhOp4oEGM3
Retrieved Bedrock AgentCore status for: aws_orchestrator_assistant


Final status: READY


#### 2.1 - Export and save outputs

Export variables to be used in clean up notebook:

In [8]:
ORCHESTRATION_ID = launch_result.agent_id
ORCHESTRATION_ARN = launch_result.agent_arn
ORCHESTRATION_NAME = agent_name

%store ORCHESTRATION_ID
%store ORCHESTRATION_ARN
%store ORCHESTRATION_NAME

Stored 'ORCHESTRATION_ID' (str)
Stored 'ORCHESTRATION_ARN' (str)
Stored 'ORCHESTRATION_NAME' (str)


### 3 - Invoking A2A agents using an orchestrator agent

Firstly, let's refresh the auth token:

In [9]:
from helpers.utils import reauthenticate_user

bearer_token = reauthenticate_user(
    COGNITO_CLIENT_ID,
    COGNITO_SECRET
)

Now, let's invoke our orchestrator to check AWS Docs, making a call to our first agent, using A2A:

In [10]:
import requests
import json
import uuid
from urllib.parse import quote

session_id = str(uuid.uuid4())
print(f'Invoking for session: {session_id}')

headers = {
    'Authorization': f'Bearer {bearer_token}',
    'Content-Type': 'application/json',
    'Accept': 'application/json',
    'X-Amzn-Bedrock-AgentCore-Runtime-Session-Id': session_id
}

prompt = {"prompt": "What is DynamoDB?"}

escaped_agent_arn = quote(ORCHESTRATION_ARN, safe='')

response = requests.post(
    f'https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{escaped_agent_arn}/invocations',
    headers=headers,
    data=json.dumps(prompt)
)

for line in response.iter_lines(decode_unicode=True):
    if line.startswith('data: '):
        data = line[6:]
        try:
            parsed = json.loads(data)
            print(parsed)
        except:
            print(data)

Invoking for session: ff813662-5eaf-436e-8cf0-7c26d3465caa
{'init_event_loop': True}
{'start': True}
{'start_event_loop': True}
{'event': {'messageStart': {'role': 'assistant'}}}
{'event': {'contentBlockStart': {'start': {'toolUse': {'toolUseId': 'tooluse_pQovp_rpQwyUkxfeNbVX3g', 'name': 'send_mcp_message', 'type': 'tool_use'}}, 'contentBlockIndex': 0}}}
{'event': {'contentBlockDelta': {'delta': {'toolUse': {'input': ''}}, 'contentBlockIndex': 0}}}
{'delta': {'toolUse': {'input': ''}}, 'current_tool_use': {'toolUseId': 'tooluse_pQovp_rpQwyUkxfeNbVX3g', 'name': 'send_mcp_message', 'input': ''}, 'agent': <strands.agent.agent.Agent object at 0xffff6de2ac30>, 'event_loop_cycle_id': UUID('1f4a96d1-25b5-4ff9-94c3-34303d6f3f01'), 'request_state': {}, 'event_loop_cycle_trace': <strands.telemetry.metrics.Trace object at 0xffff6d2fdbe0>, 'event_loop_cycle_span': _Span(name="execute_event_loop_cycle", context=SpanContext(trace_id=0x69252bab2bd9c12d639ab7f538b8947b, span_id=0x82954e90b88c3c54, tra

In [11]:
import uuid

session_id = str(uuid.uuid4())
print(f'Invoking for session: {session_id}')

headers = {
    'Authorization': f'Bearer {bearer_token}',
    'Content-Type': 'application/json',
    'Accept': 'application/json',
    'X-Amzn-Bedrock-AgentCore-Runtime-Session-Id': session_id
}

prompt = {"prompt": "Give me the latest published blog for Bedrock AgentCore?"}

escaped_agent_arn = quote(ORCHESTRATION_ARN, safe='')

response = requests.post(
    f'https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{escaped_agent_arn}/invocations',
    headers=headers,
    data=json.dumps(prompt)
)

for line in response.iter_lines(decode_unicode=True):
    if line.startswith('data: '):
        data = line[6:]
        try:
            parsed = json.loads(data)
            print(parsed)
        except:
            print(data)

Invoking for session: 10461e96-d545-4c1d-8cd2-19eb961eb7b0
{'init_event_loop': True}
{'start': True}
{'start_event_loop': True}
{'event': {'messageStart': {'role': 'assistant'}}}
{'event': {'contentBlockDelta': {'delta': {'text': "I'll search for the"}, 'contentBlockIndex': 0}}}
{'data': "I'll search for the", 'delta': {'text': "I'll search for the"}, 'agent': <strands.agent.agent.Agent object at 0xffff87a244d0>, 'event_loop_cycle_id': UUID('bdb702cd-7925-4dd9-905e-3fa7977bc6db'), 'request_state': {}, 'event_loop_cycle_trace': <strands.telemetry.metrics.Trace object at 0xffff86e59df0>, 'event_loop_cycle_span': _Span(name="execute_event_loop_cycle", context=SpanContext(trace_id=0x69252bc4460f886d03b263ef002feb23, span_id=0x66f1c4ae298a4d4d, trace_flags=0x01, trace_state=[], is_remote=False))}
{'event': {'contentBlockDelta': {'delta': {'text': ' latest'}, 'contentBlockIndex': 0}}}
{'data': ' latest', 'delta': {'text': ' latest'}, 'agent': <strands.agent.agent.Agent object at 0xffff87a244

Congratulations, you have deployed the complete solution, using A2A protocol on Amazon AgentCore Runtime.